## Análisis para Generar Rutas

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo - DNP
#os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")

# Establecer el directorio de trabajo - Remoto
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")


# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. 
nombre_archivo = "2024 11 01 - Base Original.xlsx"

data_arco = pd.read_excel(nombre_archivo)

In [17]:
data_arco.head()

,Código,Sector,Entidad,Nombre del Instrumento,Descripcion,Página Web,Antigüedad,Iteración,Emprendedores,Mipymes,...,1. Señale cuál o cuáles de las siguientes brechas,2. ¿En el diseño del instrumento se analizó el posible impacto diferenciado?,3. ¿El instrumento incorpora un enfoque interseccional?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro,Estado
0,1794,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Plan de Educación Financiera,A partir del modelo pedagógico de la entidad p...,https://www.colpensiones.gov.co/educacion/,5.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
1,1800,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Programa Beps,Los requisitos para vincularse a Hoy y Mañana ...,https://www.colpensiones.gov.co/beps/,9.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
2,1797,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Silverexpo. Feria de entretenimiento y bienest...,La feria Silverexpo se realiza en alianza con ...,www.colpensiones.gov.co,4.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
3,3689,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Adecuación de tierras,La adecuación de tierras es el servicio públi...,https://www.adr.gov.co/atencion-y-servicios-a-...,8.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,Si,No,Si,116422783387,0,0,0,0,0,Actualizado
4,1734,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Fortalecimiento a la prestación del servicio p...,MEJORAR LA TRANSFERENCIA Y APROPIACIÓN DEL CO...,https://www.adr.gov.co/atencion-y-servicios-a-...,4.0,Primera iteración 2024 - Mes de reporte: Septi...,No,No,...,No,No,No,122305692809,0,0,0,0,0,Desactualizado


In [18]:
# Get data information
data_arco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 71 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Código                                                                        419 non-null    int64         
 1   Sector                                                                        418 non-null    object        
 2   Entidad                                                                       419 non-null    object        
 3   Nombre del Instrumento                                                        419 non-null    object        
 4   Descripcion                                                                   419 non-null    object        
 5   Página Web                                                                    419 non-null  

In [19]:
# Selecting the relevant features for recommendation - Usuarios completos
selected_features = ['Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 'Gobierno ',
                 'Academia',
                 'Entidades de soporte',
                 'Personas naturales']
print(selected_features)

"""
# Selecting the relevant features for recommendation
selected_features = [
                 #'Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 #'Gobierno ',
                 #'Academia',
                 #'Entidades de soporte',
                 #'Personas naturales'
                 ]
print(selected_features)
"""

['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales']


"\n# Selecting the relevant features for recommendation\nselected_features = [\n                 #'Emprendedores',\n                 'Mipymes',\n                 'Grandes empresas',\n                 #'Gobierno ',\n                 #'Academia',\n                 #'Entidades de soporte',\n                 #'Personas naturales'\n                 ]\nprint(selected_features)\n"

In [20]:
# Replacing the null valuess with null string
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

In [21]:
# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Entidades de soporte']+ ' ' + data_arco['Personas naturales']
combined_features

"""
# combining all the 5 selected features - Con los usuarios que presentan mayor correlación
combined_features = data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] 
combined_features
"""

"\n# combining all the 5 selected features - Con los usuarios que presentan mayor correlación\ncombined_features = data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] \ncombined_features\n"

## Construyendo el sistema de recomendación

In [22]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [23]:
print(feature_vectors)

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 1)	0.1621281858740218
  (3, 0)	0.9867697053239923
  (4, 1)	0.1621281858740218
  (4, 0)	0.9867697053239923
  (5, 1)	0.5945097901233671
  (5, 0)	0.8040883716653723
  (6, 1)	0.1621281858740218
  (6, 0)	0.9867697053239923
  (7, 1)	0.3668347714227148
  (7, 0)	0.9302861121586437
  (8, 1)	0.3668347714227148
  (8, 0)	0.9302861121586437
  (9, 1)	0.5945097901233671
  (9, 0)	0.8040883716653723
  (10, 1)	0.3668347714227148
  (10, 0)	0.9302861121586437
  (11, 1)	0.5945097901233671
  (11, 0)	0.8040883716653723
  (12, 1)	0.3668347714227148
  (12, 0)	0.9302861121586437
  (13, 1)	0.3668347714227148
  (13, 0)	0.9302861121586437
  :	:
  (405, 1)	0.1621281858740218
  (405, 0)	0.9867697053239923
  (406, 1)	0.1621281858740218
  (406, 0)	0.9867697053239923
  (407, 1)	1.0
  (408, 0)	1.0
  (409, 1)	0.3668347714227148
  (409, 0)	0.9302861121586437
  (410, 1)	0.795857526821466
  (410, 0)	0.6054839362044375
  (411, 1)	1.0
  (412, 1)	0.9266257884890856
  (412, 0)	0.3759

In [24]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors, feature_vectors)

In [25]:
print(similarity)

[[1.         1.         1.         ... 0.60548394 0.60548394 0.98676971]
 [1.         1.         1.         ... 0.60548394 0.60548394 0.98676971]
 [1.         1.         1.         ... 0.60548394 0.60548394 0.98676971]
 ...
 [0.60548394 0.60548394 0.60548394 ... 1.         1.         0.72650414]
 [0.60548394 0.60548394 0.60548394 ... 1.         1.         0.72650414]
 [0.98676971 0.98676971 0.98676971 ... 0.72650414 0.72650414 1.        ]]


In [26]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = data_arco['Nombre del Instrumento'].tolist()
print(list_of_all_titles)

['Plan de Educación Financiera', 'Programa Beps', 'Silverexpo. Feria de entretenimiento y bienestar para Personas Mayores', 'Adecuación de tierras', 'Fortalecimiento a la prestación del servicio público de extensión Agropecuaria', 'FORTALECIMIENTO DE LA GESTIÓN Y APROPIACIÓN DEL CONOCIMIENTO TÉCNICO DE  LOS PROCESOS PRODUCTIVOS AGROPECUARIOS Y RURALES, EN LOS  PRODUCTORES Y LAS A', 'FORTALECIMIENTO DE LA GESTIÓN Y APROPIACIÓN DEL CONOCIMIENTO TÉCNICO DE LOS PROCESOS PRODUCTIVOS AGROPECUARIOS Y RURALES', 'IMPLEMENTACIÓN DE ESTRATEGIAS DE FORTALECIMIENTO Y APOYO COMERCIAL PARA ORGANIZACIONES RURALES A NIVEL NACIONAL', 'Proyectos Integrales de Desarrollo Agropecuario y Rural (PIDAR)', 'Obras por Impuestos', 'Asistencias técnicas realizadas', 'Escuela Virtual de Seguridad Vial', 'ORIENTACIONES TÉCNICAS PARA LA ELABORACIÓN DE PLANES LOCALES DE SEGURIDAD VIAL', 'Red de Observatorios Territoriales de Seguridad Vial (ROT)', 'SEGUIMIENTO A LA IMPLEMENTACIÓN DE LA RUTA DE ATENCIÓN INTEGRAL A VIC

In [49]:
# getting the movie name from the user
instrument_name = input(' Introducir el nombre del instrumento : ')
#instrument_name = 'Información estadística de la temática económica'


In [50]:
# finding the close match for the instrument name given by the user
find_close_match = difflib.get_close_matches(instrument_name, list_of_all_titles)
print(find_close_match)

['Emprendetón', 'Fondo Emprender']


In [51]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Ordenar los instrumentos según similitud con el nombre escogido
instrumentos_similares = sorted(
    list_of_all_titles,
    key=lambda x: calculate_similarity(instrument_name, x),
    reverse=True
)[:5]


In [52]:
print(instrumentos_similares)

['Emprendetón', 'Fondo Emprender', 'Emprende con Ciencia', 'Emprendimiento- SENA', 'Prueba Emprende NS']
